<a href="https://colab.research.google.com/github/jdmartinev/ST1613-AppliedML-/blob/main/Semana03/Regresi%C3%B3n_log%C3%ADstica_churn_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1d6iPtgmKM5N2mtrWFb-SzLdGCwLWmxF9' -O data.csv

--2023-04-20 13:28:04--  https://docs.google.com/uc?export=download&id=1d6iPtgmKM5N2mtrWFb-SzLdGCwLWmxF9
Resolving docs.google.com (docs.google.com)... 173.194.214.101, 173.194.214.138, 173.194.214.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.214.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/c0tfolcti8g1oidu8uk0qn1lgsn4fqjn/1681997250000/15315348669826032119/*/1d6iPtgmKM5N2mtrWFb-SzLdGCwLWmxF9?e=download&uuid=8a50bad0-7726-4f24-bf22-2da5cd083706 [following]
--2023-04-20 13:28:04--  https://doc-0g-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/c0tfolcti8g1oidu8uk0qn1lgsn4fqjn/1681997250000/15315348669826032119/*/1d6iPtgmKM5N2mtrWFb-SzLdGCwLWmxF9?e=download&uuid=8a50bad0-7726-4f24-bf22-2da5cd083706
Resolving doc-0g-6c-docs.googleusercontent.com (doc-0g-6c-docs.googleusercontent.com)... 173.194.218.132, 2607:

In [ ]:
import sys
import time

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("code/.")

#import mglearn
from IPython.display import display
#from plotting_functions import *


# Preprocessing and pipeline
from sklearn.impute import SimpleImputer
from scipy.stats import reciprocal

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
)
pd.set_option("display.max_colwidth", 200)

In [ ]:
#Cargamos los datos y procesamos algunas columnas
df = pd.read_csv('data.csv')

# Modificamos la variable 'TotalCharges' a tipo numérico
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

# Inlcuir guiones bajos en valores como No internet service
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.drop('customerid',axis=1,inplace=True)

#Separar los datos en entrenamiento, validación y test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=1)

#Definir características y variable objetivo
X_train = train_df.drop(columns=["churn"])
y_train = train_df["churn"]

X_test = test_df.drop(columns=["churn"])
y_test = test_df["churn"]

#Índices de las columnas numéricas y categ+oricas
cat_cols = X_train.select_dtypes(include=object).columns
num_cols = X_train.select_dtypes(include=np.number).columns


In [ ]:
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)


In [ ]:
#Definimos el clasificador base
logreg_base = LogisticRegression()
clf_logreg = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", logreg_base)])

param_grid = {
    'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'classifier__C': np.logspace(-4, 4, 20),
    'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'],
}

search_logreg = GridSearchCV(clf_logreg, param_grid, scoring='accuracy', n_jobs=-1)

search_logreg.fit(X_train, y_train)
print(search_logreg)
print(search_logreg.best_params_)

# Obtengamos el accuracy para la regresión logística 
print('Modelo regresión logística')
print(f'Accuracy: {search_logreg.score(X_test, y_test)}')

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
900 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['gender', 'partner', 'dependents', 'phoneservice...
       4.83293024e-03, 1.27427499e-02, 3.359818

In [ ]:
#Definimos el clasificador con características polinómicas
logreg_base = LogisticRegression()
clf_logreg_poly = Pipeline(steps=[("preprocessor", preprocessor), ('poly', PolynomialFeatures(include_bias=False)),("classifier", logreg_base)])

param_grid = {
    'poly__degree': list(range(2, 4)),
    'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'classifier__C': np.logspace(-4, 4, 20),
    'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'],
}

search_logreg_poly = GridSearchCV(clf_logreg_poly, param_grid, scoring='accuracy', n_jobs=-1)
search_logreg_poly.fit(X_train, y_train)
print(search_logreg_poly)
print(search_logreg_poly.best_params_)
# Obtengamos el accuracy para la regresión logística con características polinómicas:
print('Modelo regresión logística')
print(f'Accuracy: {search_logreg_poly.score(X_test, y_test)}')

In [ ]:
#Definimos el clasificador basado en redes neuronales
mlp_base = MLPClassifier()
clf_mlp = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", mlp_base)])
param_grid = {
    "classifier__hidden_layer_sizes": [(50, 50), (100, 100), (50, 100, 50)],
    "classifier__activation": ["relu", "tanh", "logistic"],
    #"solver": ["adam", "sgd"],
    "classifier__alpha": np.logspace(-5, 3, 9),
    "classifier__learning_rate": ["constant", "invscaling", "adaptive"],
    "classifier__max_iter": [200, 500, 1000]
}
search_mlp = GridSearchCV(clf_mlp, param_grid, scoring='accuracy', n_jobs=-1)
search_mlp.fit(X_train, y_train)
print(search_mlp)
print(search_mlp.best_params_)
# Obtengamos el accuracy de prueba para el MLP:
print('Modelo regresión logística')
print(f'Accuracy: {search_mlp.score(X_test, y_test)}')